# Домашнее задание

### 1. Реализовать персональный топ  - принимает на вход возраст и локацию, на выходе персональный топ   - 1 балл

Персональный топ - это топ товаров по похожим возрасту/интересам/локации. Как сделать? Разбить на сегменты по выбраным признакам. Топ делать по книгам с хорошим средним рейтингом.

### 2. На основе метода кластеризации похожих пользователей построить рекомендации (Слайд 27) - 3 балла

Нужно топ-10 рекомендаций с самой высокой оценкой. Считаем среднюю оценку для каждой книги по кластеру и выводим топ-10 книг.

### 3. Совстречаемость - 3 балла

В совстречаемости также учитывать оценки. Вес пары книг встретившихся у пользователя - полусумма их оценок.

### 4. Коллаборативная фильтрация - 3 балла

Коллаборативную фильтрацию реализовывать как на слайде 51 презентации, посоветовав каждому пользователю топ-10 книг с самой высокой оценкой. Сделать рекомендации User-based и Item-based и сравнить.

Если совсем сложно - можно сделать как в семинарской части, поставив оценку "0", если рейтинг < 5 и "1" - в противном случае. Тогда максимум за это - 1 балл. Реализовать U2I и I2I рекомендации.

### Примечание:

Так как пользователей много - можно зафиксировать несколько произвольных и для них уже составлять рекомендации

Работоспособность I2I можно проверять на известных книгах (Гарри Поттер, Властелин Колец, Интервью с вампиром, Код-Да-Винчи, Маленький Принц)

Рейтинг книг обязательно нужно учитывать

Не забываем также предобработать данные - выкинуть выбросы-пользователей и выбросы-книги.

Выводить в качестве рекомендаций лучше названия книг, картинки (если они есть) и соответствующие метрики близости.

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import re

from io import BytesIO
from PIL import Image
import requests
from scipy.sparse import coo_matrix, csr_matrix

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

In [108]:
# books = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2022_Spring/main/lecture09/HW/BX-Books.csv')
#
# interactions = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2022_Spring/main/lecture09/HW/BX-Book-Ratings.csv',
#                            sep=";", encoding = "ISO-8859-1")
#
# users = pd.read_csv('https://raw.githubusercontent.com/vadim0912/MLIntro2022_Spring/main/lecture09/HW/BX-Users.csv',
#                     delimiter=';', encoding = 'ISO-8859-1')

books=pd.read_csv("BX-Books.csv.gz")
interactions=pd.read_csv("BX-Book-Ratings.csv.gz", sep=";", encoding = "ISO-8859-1")
users=pd.read_csv("BX-Users.csv.gz",sep=";",encoding="ISO-8859-1")

C:\Users\denok\AppData\Local\Temp\ipykernel_19500\3190690250.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv("BX-Books.csv.gz")


In [109]:
interactions = interactions[interactions["Book-Rating"] != 0]

In [110]:
interactions.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6


In [111]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [112]:
books_meets = interactions.groupby("ISBN")["User-ID"].count().reset_index().rename(columns={"User-ID": "user_num"})

In [113]:
user_meets = interactions.groupby("User-ID")["ISBN"].count().reset_index().rename(columns={"ISBN": "books_num"})

In [114]:
interactions = interactions.merge(books_meets, on=["ISBN"]).merge(user_meets, on=["User-ID"])

In [115]:
interactions = interactions[(interactions["user_num"] > 5) &
                            (interactions["books_num"] > 5) &
                            (interactions["books_num"] < 200)]

In [116]:
interactions = interactions.merge(books[["ISBN", "Image-URL-M", "Book-Title"]].rename(
    columns={"Image-URL-M": "picture_url"}), on=["ISBN"])

In [117]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

In [118]:
interactions["product_id"] = le.fit_transform(interactions["ISBN"])
interactions["vid"] = le.fit_transform(interactions["User-ID"])

In [119]:
interactions.head()

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,product_id,vid
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,3445
1,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,5248
2,66483,3404139178,10,8,83,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,2622
3,276866,3404139178,9,8,11,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,10887
4,106534,3404139178,6,8,6,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,4228


In [120]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [121]:
csr_rates = coo_matrix((interactions["Book-Rating"], (interactions["vid"], interactions["product_id"])),
                            shape=(len(set(interactions["vid"])), len(set(interactions["product_id"]))))

In [122]:
for i, j in interactions[["product_id", "Book-Title"]].drop_duplicates().values:
    if "David Copperfield" in j:
        print("idx:", i, "\tBook Title:", j)

idx: 10174 	Book Title: David Copperfield (Wordsworth Classics)
idx: 1074 	Book Title: David Copperfield (Penguin Classics)


### 1) Персональный топ

In [123]:
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [124]:
class person_top():
    def __init__(self, users, interactions, Age = None, location = None):
        self.users = users.copy()
        self.interactions = interactions.copy()
        self.Age = Age
        self.city = location.split(sep = ',')[0]
        self.state = location.split(sep = ',')[1]
        self.country = location.split(sep = ',')[2]
        self.users['measure'] = np.zeros_like(self.users.Location.values)

        # штраф за разные города
        self.users.measure += 1 * (self.users.Location.str.split(pat=',', expand=True)[0] != self.city).values

        # штраф за разные штаты
        self.users.measure += 2 * (self.users.Location.str.split(pat=',', expand=True)[1] != self.state).values

        # штраф за разные страны
        self.users.measure += 3 * (self.users.Location.str.split(pat=',', expand=True)[2] != self.country).values

        if self.Age is not None:
            # штраф за разницу в возрасте
            self.users.measure[self.users.Age.notna().values] +=\
                                           np.abs(self.users.Age.values[self.users.Age.notna().values] - self.Age)
            self.users.measure[self.users.Age.isna().values] +=\
                                                            self.users.Age[self.users.Age.notna().values].median()
        self.users['weights'] = self.users.sort_values('measure').index.to_numpy()

        # чем более похожий читатель, тем больше вес
        self.interactions['weights'] = 1 / (self.users.iloc[interactions['User-ID'].values].weights.values + 100)
        self.interactions['Book-Rating'] -= 5
        self.interactions['rating_new'] = self.interactions.weights.values *\
                                          self.interactions['Book-Rating'].values

        info = self.interactions[{'ISBN', 'picture_url', 'Book-Title', 'product_id'}].drop_duplicates('product_id')

        rating = self.interactions.groupby('product_id')['rating_new'].sum()

        # книги с рейтингом
        self.library = info.merge(rating, on = ['product_id']).sort_values('rating_new', ascending=False)


    def get_rec(self, top=30):
        for idx, chunk in enumerate(chunks(self.library['picture_url'].values[:top], 5)):
            fig = plt.figure(figsize=(20, 4))
            for n, i in enumerate(chunk):
                try:
                    r = requests.get(i)
                    im = Image.open(BytesIO(r.content))
                except:
                    print('Something went wrong with {url}'.format(url=i))
                    continue
                a = fig.add_subplot(1, 5, n+1)
                plt.imshow(im)
                plt.axis('off')
            plt.show()

In [125]:
person = person_top(users=users, interactions=interactions, Age=10, location='nyc, new york, usa')

C:\Users\denok\AppData\Local\Temp\ipykernel_19500\1740273023.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.users.measure[self.users.Age.notna().values] +=\
C:\Users\denok\AppData\Local\Temp\ipykernel_19500\1740273023.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.users.measure[self.users.Age.isna().values] +=\
C:\Users\denok\AppData\Local\Temp\ipykernel_19500\1740273023.py:34: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  info = self.interactions[{'ISBN', 'picture_url', 'Book-Title', 'product_id'}].drop_duplicates('product_id')


In [126]:
person.get_rec(top=10)

Something went wrong with http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0441735762.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0812513738.01.MZZZZZZZ.jpg


<Figure size 2000x400 with 0 Axes>

Something went wrong with http://images.amazon.com/images/P/0060248025.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0812511816.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0812517725.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0345335082.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0345253736.01.MZZZZZZZ.jpg


<Figure size 2000x400 with 0 Axes>